In [215]:
#Run this code to generate a graph of path
import random
import numpy as np
num_cities = 10
# Generate random positions for cities
positions = np.random.rand(num_cities, 2) * 100  # Random positions in a 100x100 area

# Calculate the distance matrix
distance_matrix = np.zeros((num_cities, num_cities))

for i in range(num_cities):
    for j in range(i + 1, num_cities):
        distance = np.linalg.norm(positions[i] - positions[j])
        distance_rounded = int(round(distance))
        distance_matrix[i, j] = distance_rounded
        distance_matrix[j, i] = distance_rounded  # Symmetric matrix

distance_matrix = distance_matrix.astype(int)
distance_matrix

array([[  0,  71,  36,  44,  54,  41,   7,  26,  67,  12],
       [ 71,   0, 106,  35,  18,  61,  68,  97,  77,  83],
       [ 36, 106,   0,  75,  88,  71,  38,  10,  79,  24],
       [ 44,  35,  75,   0,  19,  56,  39,  67,  46,  55],
       [ 54,  18,  88,  19,   0,  51,  50,  79,  64,  66],
       [ 41,  61,  71,  56,  51,   0,  45,  62,  97,  49],
       [  7,  68,  38,  39,  50,  45,   0,  29,  60,  16],
       [ 26,  97,  10,  67,  79,  62,  29,   0,  75,  14],
       [ 67,  77,  79,  46,  64,  97,  60,  75,   0,  71],
       [ 12,  83,  24,  55,  66,  49,  16,  14,  71,   0]])

In [216]:

GENES = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
class Individual:
    #an individual has chromosome, fitness and start path
    def __init__(self, chromosome, start_point):
        self.chromosome = chromosome
        self.fitness_score = self.fitness()
        self.start_point = start_point
    def fitness(self):
        fitness = 0
        idx = 0
        while idx != len(GENES)-1:
            now = self.chromosome[idx]
            next = self.chromosome[idx + 1]
            fitness += distance_matrix[now][next]
            idx += 1               
        return fitness
    @classmethod
    def created_genes(self, start_point):
        genes = [start_point]
        candidate = [x for x in GENES if x != start_point]
        random.shuffle(candidate)
        genes.extend(candidate)
        genes.append(start_point)
        return genes
    def mutated_genes(self):
        while True:
            idx1 = random.randint(1, 9)
            idx2 = random.randint(1, 9)
            if idx1 != idx2:
                x = self.chromosome[idx1]
                self.chromosome[idx1] = self.chromosome[idx2]
                self.chromosome[idx2] = x
                break
        return self.chromosome
            
    #generate different number
    def mate(self, par2):
        prob = random.random()
        #crossover: [1, 2, 4, 3, 5, 6, 7, 9, 8, 1]  and [1, 4, 2, 5, 3, 8, 9, 7, 1]
        if prob <0.85:
            child = [-1] * 11
            child[0] = self.start_point
            child[len(child)-1] = self.start_point
            while True:
                idx = random.randint(1, 8)
                l = random.randint(2, 5)
                if l + idx <= 8:
                    break
            child[idx:idx+l] = self.chromosome[idx:idx+l]
            remain_par2 = [i for i in par2.chromosome if i not in child[idx:idx+l] and i != self.start_point]
            idx_par2 = 0
            for idx in range(1, len(child) - 1):
                if child[idx] == -1:
                    child[idx] = remain_par2[idx_par2]
                    idx_par2 += 1
        else:
            child = self.mutated_genes()
        return Individual(child, self.start_point)
      

In [235]:
POP = 1000
population = []
generation = 1
stop = False
stop_count = 50
start = int(input("You start from: "))
min_fitness = 10000
for _ in range(POP):
    x = Individual.created_genes(start)
    population.append(Individual(x, start))
while not stop:
    population = sorted(population, key=lambda x: x.fitness_score)
    #if fitness_score is the same after 50 generation, we believe this is the best result and stop running.
    if population[0].fitness_score == min_fitness:
        stop_count -= 1
    else:
        stop_count = 50
    if stop_count == 0:
        stop = True
    
    min_fitness = population[0].fitness_score
    new_population = []
    old_generation_num = int(POP/10) #take 10% of best old population - elitism
    new_population.extend(population[:old_generation_num])
    parents_num = len(population[old_generation_num:])
    for _ in range(parents_num):
        par1 = random.choice(population[:POP//2])
        par2 = random.choice(population[:POP//2])
        child = par1.mate(par2)
        new_population.append(child)
    population = new_population
    print(f"Generation: {generation}\tPath: {population[0].chromosome}\tFitness: {population[0].fitness_score}")
    generation += 1
print(f"Generation {generation-1} produces the best path with fitness score {population[0].fitness_score}")
    

Generation: 1	Path: [9, 6, 0, 7, 2, 5, 1, 3, 4, 8, 9]	Fitness: 309
Generation: 2	Path: [9, 6, 5, 0, 7, 2, 8, 3, 4, 1, 9]	Fitness: 300
Generation: 3	Path: [9, 5, 1, 4, 3, 8, 6, 0, 7, 2, 9]	Fitness: 296
Generation: 4	Path: [9, 6, 0, 3, 7, 5, 1, 4, 2, 8, 9]	Fitness: 275
Generation: 5	Path: [9, 6, 3, 0, 7, 5, 1, 4, 2, 8, 9]	Fitness: 275
Generation: 6	Path: [9, 6, 0, 7, 2, 5, 1, 4, 3, 8, 9]	Fitness: 274
Generation: 7	Path: [9, 6, 0, 7, 2, 5, 3, 4, 1, 8, 9]	Fitness: 274
Generation: 8	Path: [9, 7, 2, 0, 8, 5, 1, 4, 3, 6, 9]	Fitness: 256
Generation: 9	Path: [9, 7, 2, 6, 0, 5, 1, 4, 3, 8, 9]	Fitness: 254
Generation: 10	Path: [9, 7, 3, 6, 0, 5, 1, 4, 2, 8, 9]	Fitness: 254
Generation: 11	Path: [9, 7, 3, 6, 0, 5, 1, 4, 2, 8, 9]	Fitness: 254
Generation: 12	Path: [9, 7, 3, 6, 0, 5, 1, 4, 2, 8, 9]	Fitness: 254
Generation: 13	Path: [9, 7, 3, 6, 0, 5, 1, 4, 2, 8, 9]	Fitness: 254
Generation: 14	Path: [9, 7, 0, 6, 3, 5, 1, 4, 2, 8, 9]	Fitness: 254
Generation: 15	Path: [9, 7, 0, 6, 3, 5, 1, 4, 2, 8, 9]	Fi